## 混合混触危険性予測（2024/2/28）

CRWのデータセットに関し、危険性の値が偏らないように工夫したものを使用。また、reasonの生成に際して「CRWデータに含まれる判定のreference」をもとにするように指示。

In [36]:
import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict_mixture_p
from src.utils.prompt import prompt_ask_for_reason_mixture_p,prompt_predict_value,mask_target_value
import re

In [37]:
#api_key
openai.api_key = "api_key"

In [42]:
#laod dataset as dict
csv_path="dataset/20240225_CRW_reactivity_sorted.csv"
df=pd.read_csv(csv_path)
df = df.iloc[:,:4]
# 正規表現パターンを使用して、'Reason'列から無意味な文字列を削除します。
# 意味のない文字列を削除するための正規表現パターン
# 不要なパターンを削除する関数を定義します。
# 不要なパターンを削除する関数を定義します。
def clean_reason(text):
    # 無意味な略称の文字列を含む部分を削除します。
    # 連続するハイフン、アスタリスク、不要な行頭の大文字アルファベット+数字、余分な改行を取り除きます。
    text = re.sub(r'^[A-Z0-9]{1,2}\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'-{5,}', '', text)
    text = re.sub(r'\*{5,}', '', text)
    text = re.sub(r'\n+', ' ', text)  # 改行を単一のスペースに置換
    return text.strip()  # 文章の開始前と終了後の空白を取り除きます。

# 'Reason'列に関数を適用します。
df['Reason'] = df['Reason'].apply(clean_reason)

df.columns = ['Substance 1', 'Substance 2', 'Result', 'Reason_provided']

chemical_records=df.to_dict(orient = "records")
chemical_records

[{'Substance 1': 'ALUMINUM OXIDE',
  'Substance 2': 'ACETYLENE',
  'Result': 'Caution',
  'Reason_provided': "Non-Redox-Active Inorganic Compounds WITH Alkynes, with Acetylenic Hydrogen:    In most cases, no evidence was found for a hazardous reaction between non-redox-active inorganic compounds and alkynes with an attached hydrogen atom. However, in some cases, a hazardous product can be formed:    The explosive compound copper acetylide (Cu2C2) can be formed when copper salts or copper oxides come into contact with acetylene (Pässler, P., et al. 2011. Acetylene. Ullmann's Encyclopedia of Industrial Chemistry. (Online))."},
 {'Substance 1': 'ALUMINUM POWDER, UNCOATED',
  'Substance 2': 'ACETYLENE',
  'Result': 'Incompatible',
  'Reason_provided': 'Metals, Elemental and Powder, Active WITH Alkynes, with Acetylenic Hydrogen:    Acetylenic compounds with replaceable acetylenically bound hydrogen atoms must be kept out of contact with copper, silver, magnesium, mercury, or alloys containi

In [43]:
#このプロンプトを変更

# 質問文を生成するための指示
#qa_gen_command = """
#Provide the quantitative reasons within 300 words so that a scientist, who does not know the compatibility or reactivity of a mixture, can predict these outcomes.
#We must quantitatively consider how the combination of substances influences their compatibility or reactivity.
#I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler compounds, contributing to higher reactivity. (qualitative discussion is practically meaningless!!!)
## Therefore, the mixture is highly reactive. (Never include the answer in the reason!!).

#Good example reasons
## Mixing sodium with water generates a significant amount of heat and hydrogen gas due to the chemical reaction, which can lead to an explosive reaction under certain conditions.
## The combination of acetone and bleach creates chloroform and other hazardous compounds due to the chemical reaction between acetone's methyl groups and sodium hypochlorite in bleach.

#Output: Reason key
#"""

qa_gen_command = """
Given the detailed information below, including a Reference, conduct a detailed and quantitative analysis within 300 words to refine the prediction of the compatibility or reactivity of a mixture on a scale from 0 to 100, where 100 indicates the highest level of hazard and 0 indicates no hazard. Utilize the Reference as a starting point to deepen your analysis, focusing on the interactions between substances, their potential reactions, and quantify their effects on the mixture's safety. Your analysis should build upon the provided information, using scientific data and principles to support a more refined prediction, avoiding broad generalizations.

#example References
- "Metals, Elemental and Powder, Active WITH Hydrocarbons, Aliphatic Unsaturated:    Powdered cobalt incandescently decomposes acetylene on contact, posing a fire hazard (J. W. Mellor, 1942. Mellor’s Comprehensive Treatise on Inorganic and Theoretical Chemistry. Vol. 14, pp. 513. Longmans, Green and Co Ltd.). Hydrogen gas may be formed in this reaction (Predicted).    Polymerization of butadiene is initiated by cobalt metal (Miller, G.H. et al. 1964. Journal of Polymer Science (Part C) 1109-1115).    Acetylenic compounds with replaceable acetylenically bound hydrogen atoms must be kept out of contact with copper, silver, magnesium, mercury, or alloys containing them, to avoid formation of explosive metal acetylides (“Chemical Intermediates”, 1972 Catalogue, 158, Tamaqua (Pa.), Air Products and Chemicals Inc., 1972; Dangerous Substances: Guidance on Fires and Spillages, London, HMSO, 1972, Sect. 1, p. 27). Metals, Elemental and Powder, Active WITH Polymerizable Compounds:    Rapid polymerization of acrylonitrile can be initiated by free radicals, such as alkali or alkali earth metals (Bradzil, J. F. 2010. Acrylonitrile. Kirk-Othmer Encyclopedia of Chemical Technology. (Online)).    Polymerization of butadiene is initiated by cobalt metal (Miller, G.H. et al. 1964. Journal of Polymer Science (Part C) 1109-1115; Miller, G. H.; Eliason, R. R.; Pritchard, G. O.; J. Polymer Science C, 1963, 1109-1115).    Polymerizable:  Materials, commonly called monomers, that have the capability to undergo thermally induced or chemically initiated radical type polymerization reactions, which could generate significant amounts of heat (up to -100 kJ/mol) and pressure due to decomposition and formation of gas byproducts.  These monomers are mainly the vinyl monomers common to the chemical industry such as styrene and its derivatives (including divinyl benzene), acrylamide and its derivatives, butadiene, acrylonitrile, vinyl acetate, and others.  Also included here are the acrylates and acrylic acid derivatives but note that these materials are also captured in this tool in a separate and distinct category due to their unique hazards and their high volume of use in the industry.    These materials typically are inhibited with low ppm levels of antioxidants (inhibitors) to prevent premature polymerization chain reactions.  Many of these inhibitors require dissolved oxygen to be effective.  Inhibitor depletion is a function of time and temperature, with higher temperatures increasing depletion rates.  These materials are known to be susceptible to destabilization due to low ppm levels of contaminants.  Radical generating contaminants such as peroxides and azides are known to initiate monomer polymerization; however, the effects of seemingly benign materials are harder to predict.  Therefore, extreme caution should be used in any contamination event. And the material should be presumed to be destabilized until testing and consultation with experts.  Uncontrolled polymerization reactions can become adiabatic and lead to a serious runaway reaction with high temperatures and pressures.  The general hazards of monomers are discussed in Frurip et al., Process Safety Progress (Vol. 14, No. 2) 1995."

#Bad example reasons
- "The molecular weight of this compound compared to simpler compounds suggests higher reactivity." (This is too general and does not leverage the provided preliminary reason or provide specific interaction data.)
- "Therefore, the mixture is highly reactive." (This conclusion is unsupported by quantitative data or specific conditions that adjust the initial assessment.)

#Good example reasons
- "Uncoated aluminum powder, being an active metal, is highly reactive with bromine, a strong oxidizing agent. This reactivity is due to the high surface area of the uncoated powder form, which facilitates rapid and intense reactions. The interaction between aluminum powder and bromine is expected to be extremely vigorous, leading to the release of a significant amount of energy, potentially resulting in ignition or explosion. Historical evidence from similar reactions, such as aluminum's violent ignition upon contact with chlorine at room temperature and its vigorous reaction with iodine in the presence of water, underscores the hazardous nature of mixing aluminum with halogenating agents. Moreover, the reactive nature of powdered aluminum with various halogenated compounds, including bromine, has been well-documented, indicating a consistent pattern of incompatibility and danger when these substances come into contact. Due to these factors, the mixture of uncoated aluminum powder and bromine is classified as incompatible, with a high likelihood of violent and hazardous reactions occurring even under ambient conditions."
- "While there is generally no evidence for hazardous reactions between non-redox-active inorganic compounds and alkynes with acetylenic hydrogen, caution is warranted for the mixture of aluminum oxide and acetylene. This precaution is due to the potential formation of explosive compounds in the presence of metal oxides, analogous to the known reaction where copper oxides in contact with acetylene can form explosive copper acetylide. Although aluminum oxide is not as reactive as copper oxide, the possibility of forming a hazardous compound through a similar mechanism cannot be entirely ruled out, especially in conditions favoring such reactions. Thus, a cautionary approach is recommended until further specific data is available."
- "Argon, a noble gas, is known for its chemical inertness due to its complete valence electron shell, which makes it unreactive with most substances, including non-redox-active inorganic compounds like aluminum oxide. There is no known evidence or documented case of a hazardous reaction between argon and aluminum oxide. Argon does not participate in chemical reactions under standard conditions, and aluminum oxide is a stable, non-reactive oxide that does not easily form compounds with inert gases. Therefore, the combination of argon and aluminum oxide is considered compatible, with no risk of hazardous interactions or reactivity under normal handling and storage conditions."
Your prediction should not only adjust the preliminary analysis but also provide a refined hazard score based on a deeper quantitative assessment.

#Output: Reason key
"""
#Consider not only the potential for a reaction but also the conditions under which the reaction is safe or hazardous. Include considerations such as concentration thresholds, reaction rates, and environmental factors that mitigate or exacerbate risks.

# 分子構造から物性を予測するための指示
#predict_command = """
#Predict the compatibility or reactivity of the following mixture.
#Consider the quantitative reasons provided and, in any case, output a prediction about the mixture's safety or potential hazards.

#Output: Result key
#"""

#predict_command = """
#Predict the compatibility or reactivity of the following mixture, using only one of the following outcomes: Compatible, Incompatible, Caution. 
#Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards.

#Output: Result key should be one of [Compatible, Incompatible, Caution].
#"""

predict_command = """
Predict the level of danger of the following mixture on a scale of 0 to 100, where 0 indicates no danger at all and 100 represents the highest level of danger. 
Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards. Provide your prediction as a numerical value reflecting the mixture's level of danger.

#Output: Result key should be a numerical value between 0 and 100.
"""

#use GPT3.5 completion
llm_ask_func=json_generate

#if gen_reason=True, generate reason and predict
#if gen_reason=False, only predict without reason (for control)
gen_reason=True
#gen_reason=False

#n回のpredictionを行う
n_trials=3

In [45]:
import traceback

completed_records = []
for record in tqdm(chemical_records):  # tqdmは進捗状況を表示
    try:
        print("Hello")
            # generate_reason_and_predict関数により、理由生成と予測実行
        completed_record = generate_reason_and_predict_mixture_p(
            substance1=record["Substance 1"],
            substance2=record["Substance 2"],
            result=record["Result"],  # resultは混合物の危険性判定の結果
            qa_gen_command=qa_gen_command,  # 理由を尋ねるための指示
            predict_command=predict_command,  # 予測を行うための指示
            llm_ask_func=json_generate,  # LLM関数
            reason_provided = record["Reason_provided"],
            gen_reason = True,  # 理由を生成するかどうか
    #            smiles=record.get("smiles", ""),  # SMILES情報（存在する場合）
            n_trials=n_trials  # 予測を試みる回数
        )
#        print(completed_record)
    except Exception as e:
        print("negative")
        traceback.print_exc()
        print(e)  # 例外が発生した場合にエラーを表示
        completed_record = record  # 例外が発生した場合は元のレコードを使用

    completed_records.append(completed_record)  # 処理されたレコードをリストに追加


  0%|                                                                                         | 0/1712 [00:00<?, ?it/s]

Hello
{'Result': 60}
{'Result': 60}


  0%|                                                                               | 1/1712 [00:03<1:37:51,  3.43s/it]

{'Result': 60}
Hello
{'Result': 88}
{'Result': 88}


  0%|                                                                               | 2/1712 [00:07<1:48:07,  3.79s/it]

...

{'Result': 65}
Hello
{'Result': 55}
{'Result': 55}


100%|███████████████████████████████████████████████████████████████████████████▉| 1711/1712 [2:20:10<00:06,  6.62s/it]

{'Result': 55}
Hello
{'Result': 10}
{'Result': 10}


100%|████████████████████████████████████████████████████████████████████████████| 1712/1712 [2:20:17<00:00,  4.92s/it]

{'Result': 10}


In [50]:
# 'predicted_results'を含むレコードのみを抜き出します。
records_with_predicted_results = [record for record in completed_records if 'predicted_results' in record]

In [52]:
len(records_with_predicted_results)

1230

In [53]:
#save
import datetime
import json

study_name="mixture_practice_w_reason_numeric_train"
now=datetime.datetime.now()
now_str=now.strftime("%Y%m%d_%H%M%S")
save_path=f"results/{study_name}_{now_str}.json"
with open(save_path, 'w', encoding = "utf-8") as f:
    json.dump(records_with_predicted_results, f, indent=4, ensure_ascii=False)